# Input & Output

IO在计算机中指Input/Output，也就是输入和输出。由于程序和运行时数据是在内存中驻留，由CPU这个超快的计算核心来执行，涉及到数据交换的地方，通常是磁盘、网络等，就需要IO接口。

IO编程中，Stream（流）是一个很重要的概念，可以把流想象成一个水管，数据就是水管里的水，但是只能单向流动。Input Stream就是数据从外面（磁盘、网络）流进内存，Output Stream就是数据从内存流到外面去。对于浏览网页来说，浏览器和新浪服务器之间至少需要建立两根水管，才可以既能发数据，又能收数据。

由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

同步和异步的区别就在于是否等待IO执行的结果。好比你去麦当劳点餐，你说“来个汉堡”，服务员告诉你，对不起，汉堡要现做，需要等5分钟，于是你站在收银台前面等了5分钟，拿到汉堡再去逛商场，这是同步IO。

你说“来个汉堡”，服务员告诉你，汉堡需要等5分钟，你可以先去逛商场，等做好了，我们再通知你，这样你可以立刻去干别的事情（逛商场），这是异步IO。

很明显，使用异步IO来编写程序性能会远远高于同步IO，但是异步IO的缺点是编程模型复杂。想想看，你得知道什么时候通知你“汉堡做好了”，而通知你的方法也各不相同。如果是服务员跑过来找到你，这是回调模式，如果服务员发短信通知你，你就得不停地检查手机，这是轮询模式。总之，异步IO的复杂度远远高于同步IO。

操作IO的能力都是由操作系统提供的，每一种编程语言都会把操作系统提供的低级C接口封装起来方便使用，Python也不例外。我们后面会详细讨论Python的IO编程接口。

# 读写文件

## 读取

- 用`with`语句可以自动的执行`f.close()`
- `f.read()`会一次性读取所有内容到内存，可以通过`size`参数来指定一次读取多少，有多少读多少，如果文件中没有内容了，就返回空字符串。
- `f.readline()`一次会读取一行，如果读到文件结束了，就返回空字符串
- `f.readlines()`会一性次把所有行都读进来，返回一个`list`

In [15]:
# 一次读入所有字符
with open('./io.ipynb', 'r') as f:
    chars = f.read()
    print('type:', type(chars), ', length: ', len(chars))

type: <class 'str'> , length:  5038


In [16]:
# 一只读取100个字符
with open('./io.ipynb', 'r') as f:
    print('first 100 chars: ', f.read(100))

first 100 chars:  {
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "IO在计算机中指Input/O


In [17]:
# 一次读入一行
with open('./io.ipynb', 'r') as f:
    print(f.readline())

{



In [18]:
# 一次读入所有行
with open('./io.ipynb', 'r') as f:
    print('total lines: ', len(f.readlines()))

total lines:  191


## 读取二进制文件

- 如果打开文件时以'b'打开，则按二进制格式读取，返回的是一个bytes流。

In [5]:
with open('./io.ipynb', 'rb') as f:
    data = f.read()
    print(len(data), type(data))
    #print(data.decode('utf-8'))

8685 <class 'bytes'>


## 读取其他编码的文本

默认情况下`open()`是以`utf-8`来打开文本文件的，但是如果我们想打开其他编码方式的文件，则需要显示指定其编码方式：

```python
f = open('gbk.txt', 'r', encoding='gbk')
```
遇到有些编码不规范的文件，你可能会遇到UnicodeDecodeError，因为在文本文件中可能夹杂了一些非法编码的字符。遇到这种情况，open()函数还接收一个errors参数，表示如果遇到编码错误后如何处理。最简单的方式是直接忽略：

```python
f = open('gbk.txt', 'r', encoding='gbk', errors='ignore')
```

## 写文件

写文件和读文件是一样的，唯一区别是调用open()函数时，传入标识符'w'或者'wb'表示写文本文件或写二进制文件：

In [19]:
f = open('/tmp/tmp.txt', 'w')
f.write('hello, world\n')
f.close()

In [20]:
!cat /tmp/tmp.txt

hello, world


可以反复调用`write()`来写入文件，但是务必要调用`f.close()`来关闭文件。

当我们写文件时，操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲的时候再慢慢写入。只有调用close()方法时，操作系统才保证把没有写入的数据全部写入磁盘。忘记调用close()的后果是数据可能只写了一部分到磁盘，剩下的丢失了。所以，还是用with语句来得保险：

In [21]:
with open('/tmp/tmp.txt', 'w') as f:
    f.write('hello, world\n')

要写入特定编码的文本文件，请给`open()`函数传入`encoding`参数，将字符串自动转换成指定编码。

细心的童鞋会发现，以`'w'`模式写入文件时，如果文件已存在，会直接覆盖（相当于删掉后新写入一个文件）。如果我们希望追加到文件末尾怎么办？可以传入`'a'`以追加（append）模式写入。

所有模式的定义及含义可以参考Python的[官方文档](https://docs.python.org/3/library/functions.html#open)。

## StringIO

很多时候，数据的读写不一定是文件，也可以在内在中读写。StringIO就是把一块内存区域当成一个文本文件来读和写，读写的数据都是`str`

In [25]:
# 向StringIO写入数据
from io import StringIO

f = StringIO()
n1 = f.write('hello')
n2 = f.write(' ')
n3 = f.write('world!')
print(n1, n2, n3)
print(f.getvalue())

5 1 6
hello world!


In [27]:
# 从StringIO读取数据
txt = '''Hello!
Hi!
GoodBye!
'''
f = StringIO(txt)
s = f.readlines()
print(s)

['Hello!\n', 'Hi!\n', 'GoodBye!\n']


## BytesIO

`StringIO`操作的只能是`str`，如果要操作二进制数据，就需要使用BytesIO

In [28]:
from io import BytesIO
f = BytesIO()
n = f.write('你好，世界'.encode('utf-8'))
print(n)
print(f.getvalue())

15
b'\xe4\xbd\xa0\xe5\xa5\xbd\xef\xbc\x8c\xe4\xb8\x96\xe7\x95\x8c'
